### Step 1 — Load + schema + basic health checks

In [ ]:
import pandas as pd
from pathlib import Path

COMBINED = Path("C:/Users/User/Desktop/ML/Project/solar-potential-analysis-github-setup/cleaned_datasets/all_cities_clean.parquet")
df = pd.read_parquet(COMBINED)

print("Shape:", df.shape)
print("\nDtypes:\n", df.dtypes)

# quick null audit
nulls = df.isna().sum().sort_values(ascending=False)
print("\nNull counts:\n", nulls[nulls>0])

# unique values for keys
print("\nCities (n):", df["City"].nunique())
print("Building types (n):", df["Assumed_building_type"].nunique())

: 

### Step 2 — Coverage by city and by building type

In [ ]:
# rows per city
city_counts = df["City"].value_counts().rename_axis("City").reset_index(name="rows")
display(city_counts.head(30)); print("TOTAL rows:", city_counts["rows"].sum())

# rows per building type 
bt_counts = df["Assumed_building_type"].value_counts(dropna=False).sort_index()
display(bt_counts.rename("rows").to_frame())

# heatmap-like pivot (counts): City × Building Type
pivot_counts = pd.pivot_table(df, index="City", columns="Assumed_building_type", values="Surface_area", aggfunc="count", fill_value=0)
pivot_counts.head(30)

### Step 3 — Domain sanity checks (no engineered features)

In [ ]:
import numpy as np

checks = {}

# 3.1 Potential_installable_area should be ≤ Surface_area (allow NaNs)
bad_area = df[(df["Potential_installable_area"].notna()) &
              (df["Surface_area"].notna()) &
              (df["Potential_installable_area"] > df["Surface_area"])]

checks["area_ratio_violations"] = len(bad_area)

# 3.2 Non-negativity
nonneg_cols = ["Surface_area", "Potential_installable_area", "Peak_installable_capacity", "Energy_potential_per_year", "Estimated_tilt"]
for col in nonneg_cols:
    checks[f"negatives_in_{col}"] = int((df[col] < 0).sum(skipna=True))

# 3.3 Tilt within [0, 90]
tilt_out = df["Estimated_tilt"].dropna()
checks["tilt_out_of_range"] = int(((tilt_out < 0) | (tilt_out > 90)).sum())

# Report
checks

In [ ]:
df = df.drop(bad_area.index)
print("Dropped", len(bad_area), "rows with invalid area ratios.")

In [ ]:
df.shape

### Step 4 — Univariate distributions (existing columns only, sampled for speed)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

NUMERIC = ["Surface_area","Potential_installable_area","Peak_installable_capacity","Energy_potential_per_year","Estimated_tilt"]

# stratified-ish sample by city (cap ~200k total)
per_city = 8000
parts = []
for c, g in df.groupby("City", sort=False):
    parts.append(g.sample(min(len(g), per_city), random_state=42))
df_s = pd.concat(parts, ignore_index=True)

print("Sampled shape:", df_s.shape)

for col in NUMERIC:
    s = df_s[col].dropna()
    plt.figure(figsize=(6,3))
    plt.hist(s, bins=60)
    plt.title(col)
    plt.xlabel(col); plt.ylabel("count")
    plt.tight_layout()
    plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

NUMERIC = ["Surface_area","Potential_installable_area","Peak_installable_capacity",
           "Energy_potential_per_year","Estimated_tilt"]

# 1) Zero share + robust quantiles
summary = []
for col in NUMERIC:
    s = df_s[col].dropna()
    zeros = int((s==0).sum())
    q = s.quantile([0.5, 0.9, 0.95, 0.99, 0.999])
    summary.append({
        "col": col,
        "n": int(s.size),
        "zeros": zeros,
        "zero_share": round(zeros/max(1,s.size), 4),
        "p50": q.loc[0.5],
        "p90": q.loc[0.9],
        "p95": q.loc[0.95],
        "p99": q.loc[0.99],
        "p999": q.loc[0.999],
        "max": s.max()
    })
pd.DataFrame(summary)

In [ ]:
# 2) Re-plot (A) clipped at 99th percentile and (B) log-scale
for col in NUMERIC:
    s = df_s[col].dropna()

    # (A) Clip view at 99th percentile so the bulk is visible
    p99 = s.quantile(0.99)
    plt.figure(figsize=(6,3))
    plt.hist(s[s<=p99], bins=60)
    plt.title(f"{col} (≤ p99)")
    plt.tight_layout(); plt.show()

    # (B) Log-view to reveal the body + tail together (skip nonpositive)
    s_pos = s[s>0]
    if len(s_pos) > 0:
        plt.figure(figsize=(6,3))
        plt.hist(np.log10(s_pos), bins=60)
        plt.title(f"{col} (log10 scale, >0 only)")
        plt.xlabel(f"log10({col})"); plt.tight_layout(); plt.show()

### Step 5 — relationships

In [ ]:
import matplotlib.pyplot as plt

# A) Potential_installable_area vs Surface_area
plt.figure(figsize=(6,5))
plt.hexbin(df_s["Surface_area"], df_s["Potential_installable_area"], gridsize=60, mincnt=1)
plt.xlabel("Surface_area"); plt.ylabel("Potential_installable_area"); plt.title("Potential vs Surface")
plt.colorbar(label="count"); plt.tight_layout(); plt.show()

# B) Peak_installable_capacity vs Potential_installable_area
plt.figure(figsize=(6,5))
plt.hexbin(df_s["Potential_installable_area"], df_s["Peak_installable_capacity"], gridsize=60, mincnt=1)
plt.xlabel("Potential_installable_area"); plt.ylabel("Peak_installable_capacity"); plt.title("Peak capacity vs Potential area")
plt.colorbar(label="count"); plt.tight_layout(); plt.show()

# C) Energy_potential_per_year vs Peak_installable_capacity
plt.figure(figsize=(6,5))
plt.hexbin(df_s["Peak_installable_capacity"], df_s["Energy_potential_per_year"], gridsize=60, mincnt=1)
plt.xlabel("Peak_installable_capacity"); plt.ylabel("Energy_potential_per_year"); plt.title("Energy vs Peak capacity")
plt.colorbar(label="count"); plt.tight_layout(); plt.show()

Potential vs Surface: tight, near-linear band from the origin → potential area scales with roof size (good). A few huge roofs pull the axes—likely legit large sites.

Peak capacity vs Potential area: again ~linear → implies an almost constant W/m² across records (expected for PV density).

Energy per year vs Peak capacity: roughly linear with some spread → different climates/tilts/shading → capacity-factor differences. Still coherent.

### Step 6 — group comparisons

In [ ]:
import matplotlib.pyplot as plt

# (recompute if you restarted)
city_counts = df["City"].value_counts().rename_axis("City").reset_index(name="rows")

# Boxplots by city (Energy_potential_per_year)
cities = city_counts.head(30)["City"].tolist()
subset = df[df["City"].isin(cities)][["City","Energy_potential_per_year"]].dropna()

plt.figure(figsize=(50,6))
subset.boxplot(by="City", column="Energy_potential_per_year", rot=90)
plt.suptitle("")
plt.title("Energy_potential_per_year by City")
plt.tight_layout(); plt.show()

# Boxplots by building type (Peak_installable_capacity)
plt.figure(figsize=(6,3))
df[["Assumed_building_type","Peak_installable_capacity"]].dropna().boxplot(
    by="Assumed_building_type", column="Peak_installable_capacity"
)
plt.suptitle("")
plt.title("Peak_installable_capacity by Building Type")
plt.tight_layout(); plt.show()

### Step 7 — Report

In [ ]:
import json, numpy as np
from pathlib import Path

def to_builtin(o):
    if isinstance(o, np.generic):
        return o.item()
    if isinstance(o, dict):
        return {to_builtin(k): to_builtin(v) for k, v in o.items()}
    if isinstance(o, (list, tuple, set)):
        return [to_builtin(x) for x in o]
    return o

# if you still have report from above:
out_path = Path("data_quality_report.json")
clean_report = to_builtin(report)
out_path.write_text(json.dumps(clean_report, indent=2))
print(f"wrote {out_path.as_posix()}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

NUMERIC = ["Surface_area","Potential_installable_area","Peak_installable_capacity",
           "Energy_potential_per_year","Estimated_tilt"]

# for huge data we use df_s made in Step 4; fall back to df if needed
X = df_s if "df_s" in globals() else df
X = X[NUMERIC + ["City","Assumed_building_type"]].copy()


**Robust univariate outliers (IQR + MAD)**

IQR rule: values outside [Q1−k·IQR, Q3+k·IQR], default k=1.5 (use 3.0 to be gentler).

MAD-z rule: |x−median| / (1.4826*MAD) > c, default c=3.5 (very robust).

For strictly-positive columns we also check log10 space to handle heavy right tails.

In [ ]:
def iqr_mask(s, k=1.5):
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1
    lo, hi = q1 - k*iqr, q3 + k*iqr
    return (s < lo) | (s > hi)

def mad_mask(s, c=3.5):
    med = s.median()
    mad = (s - med).abs().median()
    if mad == 0 or np.isnan(mad):
        return pd.Series(False, index=s.index)
    z = (s - med).abs() / (1.4826 * mad)
    return z > c

def positive_log_mask(s, rule="iqr", k=1.5, c=3.5):
    # only consider strictly positive values for log check
    mpos = s > 0
    s_pos = np.log10(s[mpos])
    if rule == "iqr":
        mask = iqr_mask(s_pos, k=k)
    else:
        mask = mad_mask(s_pos, c=c)
    out = pd.Series(False, index=s.index)
    out.loc[mpos] = mask.values
    return out

positive_cols = ["Surface_area","Potential_installable_area","Peak_installable_capacity","Energy_potential_per_year"]
tilt_col = "Estimated_tilt"

uni_out = {}

for col in NUMERIC:
    s = X[col].dropna()
    m_iqr = iqr_mask(s, k=1.5)
    m_mad = mad_mask(s, c=3.5)

    # add log check for positive cols
    if col in positive_cols:
        m_log = positive_log_mask(s, rule="iqr", k=1.5)
        m_log_mad = positive_log_mask(s, rule="mad", c=3.5)
    else:
        m_log = pd.Series(False, index=s.index)
        m_log_mad = pd.Series(False, index=s.index)

    uni_out[col] = {
        "n": int(s.size),
        "iqr_outliers": int(m_iqr.sum()),
        "mad_outliers": int(m_mad.sum()),
        "log_iqr_outliers": int(m_log.sum()),
        "log_mad_outliers": int(m_log_mad.sum()),
        "p99": float(s.quantile(0.99)),
        "p999": float(s.quantile(0.999)),
        "max": float(s.max())
    }

pd.DataFrame(uni_out).T.sort_index()


**Group-wise outliers (by City and by Building Type)**

This checks whether some values are outliers within their city/type (fairer when cities differ in scale).

In [ ]:
def group_outlier_counts(frame, group_col, val_col, rule="iqr", k=1.5, c=3.5, log_for_positive=True):
    rows = []
    pos = val_col in positive_cols
    for g, sub in frame[[group_col, val_col]].dropna().groupby(group_col, sort=False):
        s = sub[val_col]
        masks = []

        if rule == "iqr":
            masks.append(iqr_mask(s, k=k))
            if log_for_positive and pos:
                masks.append(positive_log_mask(s, rule="iqr", k=k))
        else:
            masks.append(mad_mask(s, c=c))
            if log_for_positive and pos:
                masks.append(positive_log_mask(s, rule="mad", c=c))

        m = masks[0]
        for mm in masks[1:]:
            m = m | mm

        rows.append({"group": g, "n": int(s.size), "outliers": int(m.sum()),
                     "share": float(m.mean())})
    out = pd.DataFrame(rows).sort_values("share", ascending=False)
    return out

# Example: by City for Energy, and by Building type for Peak capacity
city_energy = group_outlier_counts(X, "City", "Energy_potential_per_year", rule="iqr", k=1.5)
bt_peak     = group_outlier_counts(X, "Assumed_building_type", "Peak_installable_capacity", rule="iqr", k=1.5)

city_energy.head(15), bt_peak


In [ ]:
def show_top_extremes(df_full, column, n=12):
    s = df_full[column].dropna()
    top_idx = s.nlargest(n).index
    cols = ["City","Assumed_building_type", column]
    display(df_full.loc[top_idx, cols])

for col in NUMERIC:
    print(f"\nTop extremes for {col}:")
    show_top_extremes(df, col, n=12)  # use full df here to see true top rows


In [ ]:
# City vs Energy on log-y
sub = df[df["City"].isin(df["City"].value_counts().head(10).index)]
plt.figure(figsize=(10,4))
sub.boxplot(by="City", column="Energy_potential_per_year", rot=45)
plt.yscale("log"); plt.suptitle(""); plt.title("Energy per year by City (log-y)")
plt.tight_layout(); plt.show()

# Building type vs Peak capacity on log-y
plt.figure(figsize=(7,3))
df[["Assumed_building_type","Peak_installable_capacity"]].dropna().boxplot(
    by="Assumed_building_type", column="Peak_installable_capacity")
plt.yscale("log"); plt.suptitle(""); plt.title("Peak capacity by Building Type (log-y)")
plt.tight_layout(); plt.show()


### Step 7b — correlation heatmaps for the specified columns (no City)

In [ ]:
# === Correlation heatmaps (EXISTING columns only) ===
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path

OUT_DIR = Path("cleaned_datasets"); OUT_DIR.mkdir(parents=True, exist_ok=True)

# Columns you specified (no City)
cols = [
    "Surface_area",
    "Potential_installable_area",
    "Peak_installable_capacity",
    "Energy_potential_per_year",
    "Assumed_building_type",  # already label-encoded
    "Estimated_tilt",
]
cols = [c for c in cols if c in df.columns]

# Use sampled df_s if it exists; otherwise lightly sample for speed
if "df_s" in globals():
    base = df_s
else:
    base = df.sample(n=min(len(df), 200_000), random_state=42)

X = base[cols].copy()
if "Assumed_building_type" in X:  # ensure numeric dtype
    X["Assumed_building_type"] = pd.to_numeric(X["Assumed_building_type"], errors="coerce")

# Correlations
corr_pearson  = X.corr(method="pearson")
corr_spearman = X.corr(method="spearman")

def plot_heatmap(corr, title, out_path):
    c = corr.copy()
    arr = c.values.astype(float)
    mask = np.triu(np.ones_like(arr, dtype=bool))         # show lower triangle
    arr_masked = np.ma.masked_where(mask, arr)

    fig, ax = plt.subplots(figsize=(1.0 + 0.7*len(c.columns), 1.0 + 0.7*len(c.columns)))
    im = ax.imshow(arr_masked, aspect='auto')

    ax.set_xticks(range(len(c.columns))); ax.set_xticklabels(c.columns, rotation=45, ha="right")
    ax.set_yticks(range(len(c.columns))); ax.set_yticklabels(c.columns)

    # annotate lower triangle
    for i in range(len(c.columns)):
        for j in range(len(c.columns)):
            if j <= i and not np.isnan(c.iat[i,j]):
                ax.text(j, i, f"{c.iat[i,j]:.2f}", ha="center", va="center", fontsize=9)

    cb = fig.colorbar(im, ax=ax, shrink=0.85); cb.ax.set_ylabel("Correlation", rotation=90, va="center")
    ax.set_title(title)
    plt.tight_layout()
    fig.savefig(out_path, dpi=220, bbox_inches="tight")
    plt.close(fig)

plot_heatmap(corr_pearson,  "Correlation (Pearson) — Existing Columns",  OUT_DIR / "_corr_SPECIFIED_pearson.png")
plot_heatmap(corr_spearman, "Correlation (Spearman) — Existing Columns", OUT_DIR / "_corr_SPECIFIED_spearman.png")

print("Saved:",
      (OUT_DIR / "_corr_SPECIFIED_pearson.png").as_posix(),
      (OUT_DIR / "_corr_SPECIFIED_spearman.png").as_posix())